In [1]:
import pandas as pd

df = pd.read_csv("meal_features_cleaned.csv")


In [2]:
import plotly.express as px

for col in ["calories", "protein_g", "carbs_g", "fat_g", "fiber_g"]:
    fig = px.histogram(df, x=col, nbins=30,
                       marginal="box",  # adds a boxplot above
                       title=f"Distribution of {col}",
                       color_discrete_sequence=["teal"])
    fig.update_layout(
        width=600,
        height=400,
        title_x=0.5,
        template="plotly_white"
    )
    fig.show()


## Summary of Nutrient Distributions

* All three nutrients — fiber, carbs, and fat — show right-skewed distributions.
* Most food items have low nutrient content, while a few are high in nutrients.
* Fiber: Majority below 1 g, few reach up to 4 g (high-fiber foods).
* Carbs: Most under 20 g, with some outliers above 60 g (high-carb foods).
* Fat: Majority below 5 g, few extend above 20 g (high-fat foods like oils).
* No major outliers beyond upper fences — data appears clean and accurate.
This distribution pattern is typical of real-world nutritional data.


In [3]:
import plotly.express as px

# Compute correlation matrix
corr = df.corr()

# Create heatmap
fig = px.imshow(
    corr,
    text_auto=True,       # shows correlation values
    color_continuous_scale="RdBu_r",  # similar to coolwarm
    title="Feature Correlation Heatmap",
    aspect="auto"
)

fig.update_layout(
    width=800,
    height=600,
    title_x=0.5,
    template="plotly_white"
)

fig.show()



###Correlation

Calories show strong positive correlation with both fat and carbohydrates, indicating these two nutrients are the main drivers of total calorie content.
Protein has a moderate correlation with calories and fat, suggesting it contributes to energy but less strongly.
Fiber is weakly correlated with other features, meaning it can be increased independently without significantly affecting calories.

In [4]:
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Select only the numeric feature columns
X_features = df[["calories", "protein_g", "carbs_g", "fat_g", "fiber_g"]].values

models = {
    "KMeans": KMeans(n_clusters=4, random_state=42),
    "Agglomerative": AgglomerativeClustering(n_clusters=4),
    "DBSCAN": DBSCAN(eps=5, min_samples=5),  # tune eps/min_samples
    "GMM": GaussianMixture(n_components=4, random_state=42)
}

results = []

for name, model in models.items():
    # Some models like GMM need a different call
    if name == "GMM":
        labels = model.fit_predict(X_features)
    else:
        labels = model.fit_predict(X_features)

    if len(set(labels)) > 1:  # avoid metric errors if DBSCAN finds only 1 cluster
        silhouette = silhouette_score(X_features, labels)
        ch_score = calinski_harabasz_score(X_features, labels)
        db_score = davies_bouldin_score(X_features, labels)
    else:
        silhouette, ch_score, db_score = None, None, None

    results.append({
        "Model": name,
        "Silhouette": silhouette,
        "Calinski-Harabasz": ch_score,
        "Davies-Bouldin": db_score
    })

df_results = pd.DataFrame(results)
print(df_results)


           Model  Silhouette  Calinski-Harabasz  Davies-Bouldin
0         KMeans    0.608297       25348.857550        0.530371
1  Agglomerative    0.541553       19977.577279        0.549851
2         DBSCAN   -0.533470         190.955125        2.409808
3            GMM    0.140817        1443.812909        4.651514


In [5]:
import plotly.express as px
from sklearn.decomposition import PCA

# Perform PCA
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_features)

# Create a list to hold all figures
figs = []

for name, model in models.items():
    labels = model.fit_predict(X_features)
    df_pca = pd.DataFrame({
        "PCA1": X_pca[:, 0],
        "PCA2": X_pca[:, 1],
        "Cluster": labels
    })

    fig = px.scatter(
        df_pca,
        x="PCA1",
        y="PCA2",
        color=df_pca["Cluster"].astype(str),
        title=f"{name} Clustering (PCA Projection)",
        color_discrete_sequence=px.colors.qualitative.Set2
    )

    fig.update_layout(
        width=600,
        height=500,
        title_x=0.5,
        template="plotly_white"
    )
    fig.show()



* KMeans and Agglomerative Clustering best capture the dataset’s structure, forming clear and interpretable clusters in the PCA space.
* DBSCAN struggles with over-fragmentation, implying non-uniform density.
GMM performs decently but produces overlapping clusters, which might complicate interpretation.
* Best choice: KMeans or Agglomerative Clustering — both provide clean separation and meaningful groupings for this dataset.

In [6]:
import plotly.graph_objects as go
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score

scores_kmeans = []
scores_agglom = []
K = range(2, 11)

for k in K:
    km = KMeans(n_clusters=k, random_state=42)
    labels_km = km.fit_predict(df)
    scores_kmeans.append(silhouette_score(df, labels_km))

    ac = AgglomerativeClustering(n_clusters=k)
    labels_ac = ac.fit_predict(df)
    scores_agglom.append(silhouette_score(df, labels_ac))

# Create interactive line plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(K), y=scores_kmeans,
    mode='lines+markers',
    name='KMeans',
    marker=dict(symbol='circle', size=8),
    line=dict(color='royalblue', width=2)
))

fig.add_trace(go.Scatter(
    x=list(K), y=scores_agglom,
    mode='lines+markers',
    name='Agglomerative',
    marker=dict(symbol='square', size=8),
    line=dict(color='firebrick', width=2)
))

fig.update_layout(
    title="KMeans vs Agglomerative Clustering (Silhouette Analysis)",
    xaxis_title="Number of Clusters (k)",
    yaxis_title="Silhouette Score",
    template="plotly_white",
    title_x=0.5,
    width=800,
    height=500
)

fig.show()


k = 4 provides a strong trade-off between cluster separation and interpretability. It maintains a good silhouette score and yields well-defined, meaningful clusters in both KMeans and Agglomerative methods.

In [7]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go

# --- Preprocessing ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)  # assuming df = your nutrition dataframe

# --- KMeans clustering with k=4 ---
kmeans = KMeans(n_clusters=4, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_scaled)

# --- Cluster Means ---
cluster_means = df.groupby('Cluster').mean().reset_index()
print("Cluster Means:\n", cluster_means)

# ======================
# 1️⃣ Donut Chart (Cluster Distribution)
# ======================
cluster_counts = df['Cluster'].value_counts().sort_index().reset_index()
cluster_counts.columns = ['Cluster', 'Count']

fig_donut = px.pie(
    cluster_counts,
    names='Cluster',
    values='Count',
    hole=0.4,
    title='Meal Cluster Distribution (KMeans k=4)',
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig_donut.update_traces(textposition='inside', textinfo='percent+label')
fig_donut.show()

# ======================
# 2️⃣ Radar Chart (Cluster Nutrition Profiles)
# ======================

# normalize cluster means for fair comparison
normalized_means = cluster_means.copy()
for col in cluster_means.columns[1:]:
    normalized_means[col] = cluster_means[col] / cluster_means[col].max()

categories = normalized_means.columns[1:]  # exclude 'Cluster'

fig_radar = go.Figure()

for i, row in normalized_means.iterrows():
    fig_radar.add_trace(go.Scatterpolar(
        r=row[categories].values,
        theta=categories,
        fill='toself',
        name=f'Cluster {int(row["Cluster"])}'
    ))

fig_radar.update_layout(
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 1])
    ),
    title="Normalized Nutrition Profile by Cluster",
    showlegend=True
)

fig_radar.show()


Cluster Means:
    Cluster    calories  protein_g    carbs_g      fat_g   fiber_g
0        0  315.236579   7.400645  43.550448  12.517694  2.010131
1        1  116.988950   4.307617  15.134606   4.368570  1.938515
2        2  207.370000  15.641100   7.609425  12.495333  0.489233
3        3   52.200990   1.835787   8.911302   0.763346  0.435346


In [8]:
import plotly.express as px
df_plot = pd.DataFrame(X_pca, columns=['PCA1', 'PCA2'])
df_plot['Cluster'] = labels
df_plot['meal_name'] = df.index  # optional, if your DataFrame has meal names
df_plot['calories'] = df['calories']

# Plot
fig = px.scatter(
    df_plot,
    x='PCA1',
    y='PCA2',
    color='Cluster',
    hover_data=['meal_name', 'calories'],
    title='PCA Visualization of Meal Clusters (Interactive)'
)
fig.show()

In [9]:
import pandas as pd
from sklearn.cluster import KMeans

# Assuming X is a DataFrame
k = 4
kmeans = KMeans(n_clusters=k, random_state=42)
labels = kmeans.fit_predict(df)

# Add cluster labels to your DataFrame
df['Cluster'] = labels

# Compute mean of each cluster
cluster_means = df.groupby('Cluster').mean()
print(cluster_means)


           calories  protein_g    carbs_g      fat_g   fiber_g
Cluster                                                       
0        398.309972   6.626752  52.030684  17.680912  1.849288
1         45.567785   1.833504   8.037928   0.671911  0.722659
2        140.400985   7.904198  13.558644   5.909829  1.018567
3        252.731716  11.608682  22.987739  12.324735  1.328984


In [11]:
import plotly.graph_objects as go

# Cluster 0 – Carb-Dense Balanced Meals (Weight Gain)
labels = ["protein_g", "carbs_g", "fat_g", "fiber_g"]
values = [6.626752, 52.030684, 17.680912, 1.849288]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=0.3)])
fig.update_layout(title_text='🍝 Carb-Dense Balanced Meals — Ideal for Weight Gain')
fig.show()

# Cluster 1 – Low-Calorie Light Foods (Weight Loss)
labels = ["protein_g", "carbs_g", "fat_g", "fiber_g"]
values = [1.833504, 8.037928, 0.671911, 0.722659]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=0.3)])
fig.update_layout(title_text='🥗 Low-Calorie Light Foods — Ideal for Weight Loss')
fig.show()

# Cluster 2 – Protein-Rich Balanced Meals (Maintenance)
labels = ["protein_g", "carbs_g", "fat_g", "fiber_g"]
values = [7.904198, 13.558644, 5.909829, 1.018567]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=0.3)])
fig.update_layout(title_text='🍛 Protein-Rich Balanced Meals — Ideal for Maintenance')
fig.show()

# Cluster 3 – High-Protein Energy Meals (Maintenance / Gain)
labels = ["protein_g", "carbs_g", "fat_g", "fiber_g"]
values = [11.608682, 22.987739, 12.324735, 1.328984]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=0.3)])
fig.update_layout(title_text='🍗 High-Protein Energy Meals — Ideal for Maintenance or Muscle Gain')
fig.show()





In [12]:
cluster_labels = {
    0: "🍝 Carb-Dense Balanced Meals — Ideal for Weight Gain",
    1: "🥗 Low-Calorie Light Foods — Ideal for Weight Loss",
    2: "🍛 Protein-Rich Balanced Meals — Ideal for Maintenance",
    3: "🍗 High-Protein Energy Meals — Ideal for Maintenance or Muscle Gain",
    4: "🍞 Protein-Rich Light Foods — Ideal for Weight Maintenance"
}

df["meal_type"] = df["Cluster"].map(cluster_labels)



###FINAL DATASET

In [13]:
df.head(5)

,calories,protein_g,carbs_g,fat_g,fiber_g,Cluster,meal_type
0,177.0,21.24,7.46,6.12,0.0,2,🍛 Protein-Rich Balanced Meals — Ideal for Main...
1,172.0,16.89,8.14,7.66,0.4,2,🍛 Protein-Rich Balanced Meals — Ideal for Main...
2,181.0,17.69,8.86,7.94,0.2,2,🍛 Protein-Rich Balanced Meals — Ideal for Main...
3,88.0,0.21,2.10,2.38,0.0,1,🥗 Low-Calorie Light Foods — Ideal for Weight Loss
4,58.0,0.90,0.27,0.00,0.0,1,🥗 Low-Calorie Light Foods — Ideal for Weight Loss


In [16]:
original_df = pd.read_csv("original.csv")

In [18]:
# Align them by index
merged_df = pd.concat([original_df[["description", "dataType"]], df], axis=1)

# Save the new dataset
merged_df.to_csv("merged.csv", index=False)

print(merged_df.head())

              description        dataType  calories  protein_g  carbs_g  \
0                 Abalone  Survey (FNDDS)     177.0      21.24     7.46   
1     Adobo, with noodles  Survey (FNDDS)     172.0      16.89     8.14   
2        Adobo, with rice  Survey (FNDDS)     181.0      17.69     8.86   
3  Agave liquid sweetener  Survey (FNDDS)      88.0       0.21     2.10   
4  Alcoholic coffee drink  Survey (FNDDS)      58.0       0.90     0.27   

   fat_g  fiber_g  Cluster                                          meal_type  
0   6.12      0.0      2.0  🍛 Protein-Rich Balanced Meals — Ideal for Main...  
1   7.66      0.4      2.0  🍛 Protein-Rich Balanced Meals — Ideal for Main...  
2   7.94      0.2      2.0  🍛 Protein-Rich Balanced Meals — Ideal for Main...  
3   2.38      0.0      1.0  🥗 Low-Calorie Light Foods — Ideal for Weight Loss  
4   0.00      0.0      1.0  🥗 Low-Calorie Light Foods — Ideal for Weight Loss  
